#### Load package

In [1]:
import pandas as pd
pd.options.mode.chained_assignment = None
import numpy as np

#### Load data

In [2]:
SNFs = pd.read_excel(r'D:\fall2020\case study\case study 1\CaseStudy_1\Data\SNFs.xlsx')

In [3]:
Episodes = pd.read_excel(r'D:\fall2020\case study\case study 1\CaseStudy_1\Data\Episodes.xlsx')

#### Data first look

In [4]:
SNFs.head()

,ProviderName,PID,ProviderPracticeAddress1,ProviderPracticeAddress2,ProviderPracticeCity,ProviderPracticeStateCode,ProviderPracticeZipCode,County
0,648 Medical Squadron,1,8006 CHENNAULT RD,STE 1,BROOKS CITY BASE,TX,78235,Bexar
1,Abilene Nursing and Rehabilitation Center,2,2630 OLD ANSON RD,NaN,ABILENE,TX,79603,Taylor
2,Abilene Regional Medical Center,3,6250 US HIGHWAY 83,NaN,ABILENE,TX,79606,Taylor
3,Abilene Regional Medical Center Skilled Nursin...,4,6250 HIGHWAY 83 84 AT ANTILLEY RD,NaN,ABILENE,TX,79606,Taylor
4,"Absolutely Angels, Inc",5,936 E PRODUCTION DR,NaN,PILOT POINT,TX,76258,Denton


In [5]:
# count the number of unique values for each column in SNFs
SNFs.nunique()

ProviderName                 1797
PID                          1813
ProviderPracticeAddress1     1728
ProviderPracticeAddress2       16
ProviderPracticeCity          507
ProviderPracticeStateCode       1
ProviderPracticeZipCode       782
County                        235
dtype: int64

In [6]:
SNFs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1813 entries, 0 to 1812
Data columns (total 8 columns):
ProviderName                 1813 non-null object
PID                          1813 non-null int64
ProviderPracticeAddress1     1813 non-null object
ProviderPracticeAddress2     17 non-null object
ProviderPracticeCity         1813 non-null object
ProviderPracticeStateCode    1813 non-null object
ProviderPracticeZipCode      1813 non-null int64
County                       1674 non-null object
dtypes: int64(2), object(6)
memory usage: 113.4+ KB


In [7]:
# percent of missingness for all columns in SNFs
count = SNFs.isna().sum()
missingness = (pd.concat([count.rename('missing_count'),
                     count.div(len(SNFs))
                          .rename('missing_ratio')],axis = 1)
             .loc[count.ne(0)])
missingness.sort_values(by = ['missing_ratio'], ascending=False)

,missing_count,missing_ratio
ProviderPracticeAddress2,1796,0.990623
County,139,0.076669


Suggest to use city instead of county since there is no missing values for city variable

#### Align SNFs with Episodes

In [4]:
# merge SNFs with Episodes and only keep SNF that has patient data in Episodes
# Reasons to do this: simplier question and, for the SNFs that do not have any patient record,
# no way to confirm that they need CCs and they need how many CCs.
common_epi_snf = SNFs.merge(Episodes, on=["PID"])
SNFs_no_episodes = SNFs[~SNFs.PID.isin(common_epi_snf.PID)]

In [9]:
SNFs_no_episodes.head()

,ProviderName,PID,ProviderPracticeAddress1,ProviderPracticeAddress2,ProviderPracticeCity,ProviderPracticeStateCode,ProviderPracticeZipCode,County
0,648 Medical Squadron,1,8006 CHENNAULT RD,STE 1,BROOKS CITY BASE,TX,78235,Bexar
1,Abilene Nursing and Rehabilitation Center,2,2630 OLD ANSON RD,NaN,ABILENE,TX,79603,Taylor
2,Abilene Regional Medical Center,3,6250 US HIGHWAY 83,NaN,ABILENE,TX,79606,Taylor
3,Abilene Regional Medical Center Skilled Nursin...,4,6250 HIGHWAY 83 84 AT ANTILLEY RD,NaN,ABILENE,TX,79606,Taylor
4,"Absolutely Angels, Inc",5,936 E PRODUCTION DR,NaN,PILOT POINT,TX,76258,Denton


In [5]:
common_snf = SNFs.merge(SNFs_no_episodes, on=["PID"])
SNFs_with_episodes = SNFs[~SNFs.PID.isin(common_snf.PID)]

In [6]:
SNFs_with_episodes.head()

,ProviderName,PID,ProviderPracticeAddress1,ProviderPracticeAddress2,ProviderPracticeCity,ProviderPracticeStateCode,ProviderPracticeZipCode,County
5,Accel at College Station,6,1500 MEDICAL AVE,NaN,COLLEGE STATION,TX,77845,NaN
7,Accel at Willow Bend,8,2620 COMMUNICATIONS PKWY,NaN,PLANO,TX,75093,Collin
8,Adora Midtown Park,9,8130 MEADOW RD,NaN,DALLAS,TX,75231,NaN
9,Advance Healthcare Solutions,10,4863 KEMP BLVD,NaN,WICHITA FALLS,TX,76308,Wichita
10,Advanced Health and Rehabilitation Center of G...,11,1201 COLONEL DR,NaN,GARLAND,TX,75043,Dallas


In [12]:
SNFs_with_episodes.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1248 entries, 5 to 1812
Data columns (total 8 columns):
ProviderName                 1248 non-null object
PID                          1248 non-null int64
ProviderPracticeAddress1     1248 non-null object
ProviderPracticeAddress2     3 non-null object
ProviderPracticeCity         1248 non-null object
ProviderPracticeStateCode    1248 non-null object
ProviderPracticeZipCode      1248 non-null int64
County                       1186 non-null object
dtypes: int64(2), object(6)
memory usage: 87.8+ KB


One problem here is the repeating observations, and there are two conditions:

1. Same SNF with two observations, one has name in upper case, one has name in lower case
2. Same SNF with similar name (has confirmed with the professor, and he said observations with the same location are actually representing the same SNF)

So I took out the repeating rows be the following steps.

In [7]:
# Aligh with unique address and city
repeat = SNFs_with_episodes[SNFs_with_episodes.duplicated(subset=['ProviderPracticeAddress1', 'ProviderPracticeCity'],keep=False)].sort_values(by ='ProviderPracticeAddress1')
repeat.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26 entries, 560 to 11
Data columns (total 8 columns):
ProviderName                 26 non-null object
PID                          26 non-null int64
ProviderPracticeAddress1     26 non-null object
ProviderPracticeAddress2     0 non-null object
ProviderPracticeCity         26 non-null object
ProviderPracticeStateCode    26 non-null object
ProviderPracticeZipCode      26 non-null int64
County                       17 non-null object
dtypes: int64(2), object(6)
memory usage: 1.8+ KB


In [8]:
common_both = SNFs_with_episodes.merge(repeat, on=["PID"])
SNFs_unique = SNFs_with_episodes[~SNFs_with_episodes.PID.isin(common_both.PID)]

In [9]:
SNFs_unique.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1222 entries, 5 to 1812
Data columns (total 8 columns):
ProviderName                 1222 non-null object
PID                          1222 non-null int64
ProviderPracticeAddress1     1222 non-null object
ProviderPracticeAddress2     3 non-null object
ProviderPracticeCity         1222 non-null object
ProviderPracticeStateCode    1222 non-null object
ProviderPracticeZipCode      1222 non-null int64
County                       1169 non-null object
dtypes: int64(2), object(6)
memory usage: 85.9+ KB


In [10]:
repeat_for_join = repeat.groupby(['ProviderPracticeAddress1','County'], as_index = False).first()

In [11]:
repeat_for_join

,ProviderPracticeAddress1,County,ProviderName,PID,ProviderPracticeAddress2,ProviderPracticeCity,ProviderPracticeStateCode,ProviderPracticeZipCode
0,101 POSEY AVE,Bosque,Goodall Witcher Nursing Facility Swing Bed,561,NaN,CLIFTON,TX,76634
1,11106 CHRISTUS HLS,Bexar,Windemere at Westover Hills,1771,NaN,SAN ANTONIO,TX,78251
2,1154 E HAWKINS PKWY,Gregg,Treviso Transitional Care,1631,NaN,LONGVIEW,TX,75605
3,1302 NOTTINGHAM ST,Walker,Focused Care At Huntsville,485,NaN,HUNTSVILLE,TX,77340
4,14100 KARISSA CT,Harris,"Bridgecrest Rehabiliation Suites, LLC",146,NaN,HOUSTON,TX,77049
5,14949 MESA DR,Harris,Fall Creek Rehabilitation And Healthcare Center,464,NaN,HUMBLE,TX,77396
6,1621 COIT RD,Collin,Landmark At Plano,802,NaN,PLANO,TX,75075
7,2000 BEAUMONT RD,Harris,Focused Care At Baytown,478,NaN,BAYTOWN,TX,77520
8,2625 BELT LINE RD,Dallas,Legend Oaks Healthcare and Rehabilitation Garland,836,NaN,GARLAND,TX,75044
9,2700 S HENDERSON BLVD,Gregg,Arbor Grace Of Kilgore,38,NaN,KILGORE,TX,75662


In [12]:
frames = [SNFs_unique, repeat_for_join]
SNFs_fixed = pd.concat(frames, sort = False)

In [13]:
# SNFs after the fixing process
# Ready to use
SNFs_fixed.nunique()

ProviderName                 1229
PID                          1235
ProviderPracticeAddress1     1233
ProviderPracticeAddress2        3
ProviderPracticeCity          433
ProviderPracticeStateCode       1
ProviderPracticeZipCode       664
County                        212
dtype: int64

One thought here:

Counts of providers for county have relative higher values, which means for the result, when assigning CCs, the counts would be higher for CCs if we go with county

In [14]:
SNFs_fixed.groupby(["ProviderPracticeCity"]).ProviderName.count().sort_values(ascending=False)

ProviderPracticeCity
HOUSTON                 70
SAN ANTONIO             61
FORT WORTH              38
DALLAS                  35
AUSTIN                  26
EL PASO                 21
CORPUS CHRISTI          16
TYLER                   12
LUBBOCK                 11
AMARILLO                11
TEMPLE                  11
WACO                    11
ARLINGTON               10
LONGVIEW                 9
PLANO                    9
ABILENE                  9
MCALLEN                  9
WICHITA FALLS            9
MESQUITE                 8
BEAUMONT                 8
RICHARDSON               7
WEATHERFORD              7
PASADENA                 7
KATY                     7
DENTON                   7
KERRVILLE                7
SAN ANGELO               6
TEXAS CITY               6
TEXARKANA                6
BROWNSVILLE              6
                        ..
DEVINE                   1
ROBERT LEE               1
FORNEY                   1
PHARR                    1
FRANKLIN                 1
PERRYTO

In [15]:
SNFs_fixed.groupby(["County"]).ProviderName.count().sort_values(ascending=False)

County
Harris         103
Dallas          76
Tarrant         70
Bexar           60
Travis          28
Collin          22
Hidalgo         20
Nueces          17
Denton          17
Bell            17
El Paso         17
McLennan        15
Fort Bend       14
Wichita         14
Brazoria        13
Smith           13
Lubbock         12
Jefferson       12
Williamson      12
Galveston       12
Gregg           12
Cameron         11
Grayson         10
Taylor          10
Cherokee         9
Montgomery       9
Ellis            9
Lavaca           8
Potter           8
Johnson          8
              ... 
Leon             1
Live Oak         1
Rains            1
Lynn             1
Martin           1
McCulloch        1
Menard           1
Mills            1
Mitchell         1
Morris           1
Newton           1
Kimble           1
Jim Hogg         1
Jack             1
Gaines           1
Dallam           1
Delta            1
Upton            1
Duval            1
Foard            1
Franklin         1
Stone

In [22]:
SNFs_fixed.to_csv('SNFs.csv', index=False)

#### Episodes

In [16]:
Episodes.nunique()

PID               1248
EpisodeID        40998
ContractType         2
AdmitDate           12
DischargeDate      534
dtype: int64

In [24]:
Episodes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40998 entries, 0 to 40997
Data columns (total 5 columns):
PID              40998 non-null int64
EpisodeID        40998 non-null int64
ContractType     40998 non-null object
AdmitDate        40998 non-null object
DischargeDate    39849 non-null datetime64[ns]
dtypes: datetime64[ns](1), int64(2), object(2)
memory usage: 1.6+ MB


Replacing wrong PID with the correct one

In [17]:
Episodes['PID'] = Episodes['PID'].replace([560, 1770, 1632, 484, 147, 463, 804, 474, 837, 757, 135, 612, 16],[561, 1771, 1631, 485, 146, 464, 802, 478, 836, 38, 138, 613, 12])

In [18]:
# convert to datetime type
Episodes['AdmitDate']= pd.to_datetime(Episodes['AdmitDate'])
Episodes['DischargeDate']= pd.to_datetime(Episodes['DischargeDate'])

In [19]:
count = Episodes.isna().sum()
missingness = (pd.concat([count.rename('missing_count'),
                     count.div(len(Episodes))
                          .rename('missing_ratio')],axis = 1)
             .loc[count.ne(0)])
missingness.sort_values(by = ['missing_ratio'], ascending=False)

,missing_count,missing_ratio
DischargeDate,1149,0.028026


Taking out rows that doesn't have a discharge date

In [21]:
Episodes = Episodes.dropna(axis=0, subset=['DischargeDate'])

Having a new column storing the time difference

In [22]:
# have a new column store time difference
Episodes['time_diff'] = Episodes['DischargeDate'] - Episodes['AdmitDate']

In [32]:
Episodes = pd.DataFrame(Episodes)

In [34]:
Episodes

,PID,EpisodeID,ContractType,AdmitDate,DischargeDate,time_diff
0,530,1,CT1,2019-12-01,2020-01-30,60 days
2,1107,3,CT2,2019-12-01,2019-12-07,6 days
3,1107,4,CT2,2019-12-01,2019-12-02,1 days
5,842,6,CT2,2019-12-01,2019-12-04,3 days
6,1178,7,CT2,2019-12-01,2019-12-17,16 days
7,857,8,CT1,2019-12-01,2019-12-23,22 days
8,1553,9,CT1,2019-12-01,2019-12-28,27 days
9,1187,10,CT1,2019-12-01,2019-12-02,1 days
10,268,11,CT1,2019-12-01,2020-01-21,51 days
11,1516,12,CT2,2019-12-01,2020-01-03,33 days


In [24]:
Episodes.min()

PID                                6
EpisodeID                          1
ContractType                     CT1
AdmitDate        2019-01-01 00:00:00
DischargeDate    2018-11-01 00:00:00
time_diff         -61 days +00:00:00
dtype: object

In [42]:
Episodes = Episodes[Episodes['time_diff'].dt.days >= 0]

In [43]:
Episodes = Episodes.drop(['time_diff'], axis = 1)

In [45]:
Episodes

,PID,EpisodeID,ContractType,AdmitDate,DischargeDate
0,530,1,CT1,2019-12-01,2020-01-30
2,1107,3,CT2,2019-12-01,2019-12-07
3,1107,4,CT2,2019-12-01,2019-12-02
5,842,6,CT2,2019-12-01,2019-12-04
6,1178,7,CT2,2019-12-01,2019-12-17
7,857,8,CT1,2019-12-01,2019-12-23
8,1553,9,CT1,2019-12-01,2019-12-28
9,1187,10,CT1,2019-12-01,2019-12-02
10,268,11,CT1,2019-12-01,2020-01-21
11,1516,12,CT2,2019-12-01,2020-01-03


In [46]:
Episodes.to_csv('Episodes.csv', index=False)

#### EDA on Contract

In [140]:
#Total number of patients based on PID and Contract type
Episodes.groupby(["PID", "ContractType"]).ContractType.count()

PID   ContractType
6     CT1               1
      CT2              65
8     CT1              25
      CT2              82
9     CT1              61
      CT2              26
10    CT2               4
11    CT1               2
      CT2              23
12    CT1              13
      CT2              19
13    CT1               1
      CT2              25
14    CT2              52
15    CT2              37
17    CT1              43
      CT2              72
18    CT2              37
19    CT2              33
20    CT1              12
      CT2              13
21    CT1               1
      CT2               5
22    CT1              87
      CT2              12
23    CT1              18
      CT2               8
25    CT1              56
      CT2              46
27    CT1               6
                     ... 
1788  CT2               7
1789  CT2               5
1790  CT2               1
1791  CT1               1
      CT2               1
1792  CT2               1
1793  CT1          

In [141]:
#Total days spent based on PID and Contract type
Episodes.groupby(["PID", "ContractType"])['time_diff'].agg('sum')

PID   ContractType
6     CT1               7 days
      CT2            1062 days
8     CT1            1047 days
      CT2            1287 days
9     CT1            1653 days
      CT2             419 days
10    CT2              56 days
11    CT1             168 days
      CT2             367 days
12    CT1             324 days
      CT2             264 days
13    CT1              91 days
      CT2             353 days
14    CT2             895 days
15    CT2             553 days
17    CT1             679 days
      CT2            1029 days
18    CT2             609 days
19    CT2             481 days
20    CT1             267 days
      CT2             271 days
21    CT1               0 days
      CT2              69 days
22    CT1            2564 days
      CT2             192 days
23    CT1             216 days
      CT2              92 days
25    CT1            1260 days
      CT2             711 days
27    CT1             150 days
                        ...   
1788  CT2           

Start from here: may be find some linear relationship based on this following chart？

In [142]:
# How many patients go to the same SNFs on the same dayunder different contract type
Episodes.groupby(["PID", "AdmitDate", "ContractType"]).ContractType.count()

PID   AdmitDate   ContractType
6     2019-01-01  CT2             11
      2019-02-01  CT2              5
      2019-03-01  CT2              7
      2019-04-01  CT1              1
                  CT2              4
      2019-05-01  CT2              4
      2019-06-01  CT2              3
      2019-07-01  CT2              2
      2019-08-01  CT2              5
      2019-09-01  CT2              6
      2019-10-01  CT2              4
      2019-11-01  CT2              3
      2019-12-01  CT2             11
8     2019-01-01  CT1              1
                  CT2             19
      2019-02-01  CT1              2
                  CT2              8
      2019-03-01  CT1              2
                  CT2              8
      2019-04-01  CT1              1
                  CT2              6
      2019-05-01  CT1              3
                  CT2              8
      2019-06-01  CT1              1
                  CT2             13
      2019-07-01  CT1              1
       

More questions:

Did not figure out how to calculate the estimate time for the two types of contract.

After figuring this out, we could add it as a feature?